In [14]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Model
from tensorflow.keras.layers import Input, Embedding, Bidirectional, LSTM, Dense

vocab_size = 2500  # 사용할 단어 개수
maxlen = 20  # Bi-LSTM에 들어갈 최대 input sequence의 수 (단어 수)

## Build the model in Functional API style

In [15]:
# input layer -> Bi-LSTM -> Dense(output)
inputs = Input(shape=(None,), dtype="int32")

x = Embedding(vocab_size, 50)(inputs) ## text -> vector (100d)
x = Bidirectional(LSTM(128))(x) # # of hidden layer node = 128 (hyper-parameter)

outputs = Dense(1, activation="sigmoid")(x) # [0, 1] ## binary classification
model = Model(inputs, outputs)
model.summary()

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, None)]            0         
                                                                 
 embedding_3 (Embedding)     (None, None, 50)          125000    
                                                                 
 bidirectional_3 (Bidirectio  (None, 256)              183296    
 nal)                                                            
                                                                 
 dense_3 (Dense)             (None, 1)                 257       
                                                                 
Total params: 308,553
Trainable params: 308,553
Non-trainable params: 0
_________________________________________________________________


## IMDB 감정분석 데이터셋 불러오기

In [16]:
(X_train, y_train), (X_test, y_test) = keras.datasets.imdb.load_data(num_words=vocab_size)

print(len(X_train), "Training sequences")
print(len(X_test), "Test sequences")

X_train = keras.preprocessing.sequence.pad_sequences(X_train, maxlen=maxlen) # zero-padding
X_test = keras.preprocessing.sequence.pad_sequences(X_test, maxlen=maxlen)

25000 Training sequences
25000 Test sequences


## Train and evaluate the model

In [17]:
# training setup
from tensorflow.keras.optimizers import SGD

## config variables
batch_size = 128
lr = 0.1
epochs = 5

# Set optimizer, loss function, metrics
optimizer = SGD(lr, momentum=0.9)
loss_fn = "binary_crossentropy" # binary-classification
metrics = ["accuracy"]

model.compile(optimizer=optimizer,
              loss=loss_fn,
              metrics=metrics)

In [19]:
model.fit(X_train, y_train,
          batch_size=128,
          epochs=5,
          verbose=1,
          validation_data=(X_test, y_test))

Epoch 1/5
196/196 [==============================] - 22s 111ms/step - loss: 0.5377 - accuracy: 0.7257 - val_loss: 0.5198 - val_accuracy: 0.7338
Epoch 2/5
196/196 [==============================] - 21s 110ms/step - loss: 0.5036 - accuracy: 0.7492 - val_loss: 0.5929 - val_accuracy: 0.6922
Epoch 3/5
196/196 [==============================] - 22s 114ms/step - loss: 0.4926 - accuracy: 0.7566 - val_loss: 0.5011 - val_accuracy: 0.7512
Epoch 4/5
196/196 [==============================] - 22s 114ms/step - loss: 0.4848 - accuracy: 0.7633 - val_loss: 0.5416 - val_accuracy: 0.7223
Epoch 5/5
196/196 [==============================] - 22s 113ms/step - loss: 0.4818 - accuracy: 0.7642 - val_loss: 0.5032 - val_accuracy: 0.7472


In [20]:
model.evaluate(X_test, y_test)

782/782 [==============================] - 8s 11ms/step - loss: 0.5032 - accuracy: 0.7472


[0.503247082233429, 0.7472000122070312]